# 1. TensorBoard
**TensorBoard** is a visualization toolkit for machine learning experimentation. TensorBoard allows tracking and visualizing metrics such as loss and accuracy, visualizing the model graph, viewing histograms, displaying images, etc.
1. `torch.utils.tensorboard.writer.SummaryWriter(log_dir=None, comment='', purge_step=None, max_queue=10, flush_secs=120, filename_suffix='')`: Writes entries directly to event files in the `log_dir` to be consumed by TensorBoard.
    - [torch.utils.tensorboard](https://pytorch.org/docs/stable/tensorboard.html#module-torch.utils.tensorboard)

In [ ]:
import torch, torchvision

batch_size = 32

# Load the dataset & preprocess
train_dataset = torchvision.datasets.MNIST(root = './datasets',
                                           train = True,
                                           transform = torchvision.transforms.Compose([
                                               torchvision.transforms.Resize((32, 32)),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)

test_dataset = torchvision.datasets.MNIST(root = './datasets',
                                          train = False,
                                          transform = torchvision.transforms.Compose([
                                              torchvision.transforms.Resize((32, 32)),
                                              torchvision.transforms.ToTensor(),
                                              torchvision.transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)

train_dataloader = torch.utils.data.DataLoader(dataset = train_dataset,
                                               batch_size = batch_size,
                                               shuffle = True)

test_dataloader = torch.utils.data.DataLoader(dataset = test_dataset,
                                              batch_size = batch_size,
                                              shuffle = True)

In [7]:
import torch.nn as nn

# Define the model
class LeNet5(nn.Module):
    def __init__(self, K):
        super(LeNet5, self).__init__()
        self.convolutional_layers = nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),
            # torch.nn.BatchNorm2d(6),
            # torch.nn.ReLU(),
            torch.nn.Sigmoid(),
            torch.nn.AvgPool2d(kernel_size = 2, stride = 2),
            torch.nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
            # torch.nn.BatchNorm2d(16),
            # torch.nn.ReLU(),
            torch.nn.Sigmoid(),
            torch.nn.AvgPool2d(kernel_size = 2, stride = 2)
        )
        self.dense_layers = nn.Sequential(
            # torch.nn.Dropout(0.2),
            # 16 * 6 * 6 = 576
            torch.nn.Linear(576, 120),
            # torch.nn.ReLU(),
            # torch.nn.Dropout(0.2),
            torch.nn.Sigmoid(),
            torch.nn.Linear(120, 84),
            # torch.nn.ReLU(),
            torch.nn.Sigmoid(),
            torch.nn.Linear(84, K)
        )
  
    def forward(self, X):
        X = self.convolutional_layers(X)
        # Flatten
        X = X.view(X.size(0), -1)
        X = self.dense_layers(X)
        return X

num_classes = 10

# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

model = LeNet5(num_classes).to(device)
print(model)

# Create a loss function for multi-class classification
criterion = torch.nn.CrossEntropyLoss()

# Create an optimizer for multi-class classification
learning_rate = 0.1
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

LeNet5(
  (convolutional_layers): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (dense_layers): Sequential(
    (0): Linear(in_features=576, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [8]:
# Initialize Tensorboard
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [9]:
# Define relevant variables for the traing task
num_epochs = 10

# Loop through data
for epoch in range(num_epochs):
    # Training
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Tensorboard
        writer.add_scalar("Loss/train", loss, epoch)
        
        # Backward & optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

writer.flush()
writer.close()

In [14]:
!tensorboard --logdir=runs
# Then go to `http://localhost:6006/#timeseries`

2024-10-22 16:36:24.301892: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-22 16:36:24.308520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 16:36:24.316404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 16:36:24.318817: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 16:36:24.324619: I tensorflow/core/platform/cpu_feature_guar

In [16]:
# Use the `%` Jupyter notebook magic function 
%load_ext tensorboard
%tensorboard --logdir=runs